# Практичне завдання з дисципліни Методи обробки природномовної інформації

Варіант 111


Завдання:

Розробити Jupyter Notebook для вирішення завдання класифікації тексту
на основі набору даних:
[link]

Забезпечити виконання наступних етапів:

1. Завантаження та попередній аналіз даних.
2. Створення тренувального та тестувального наборів даних.
3. Попередню оцінку точності моделі за допомогою методу Naive Bayes.
4. Вирішення задачі класифікації за допомогою методу [method].
5. Графічне відображення тренування моделі.
6. Порівняння результатів, отриманих у п.п. 3,4
7. Висновки.


# 1. Завантаження та попередній аналіз даних.

In [1]:
# імпортуємо необхідні бібліотеки
import tensorflow as tf, requests as rqst, io
import numpy as np
import pandas as pd
import random
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

2021-12-13 12:00:46.364048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-13 12:00:46.364070: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# 2. Створення тренувального та тестувального наборів даних.


In [2]:
file = open('train.csv','r')
new_file = open('train_df.csv', 'w')
for i, line in enumerate(file):
  if i>930 and i<940:
    continue
  new_file.write(line)

In [3]:
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('valid.csv')
train_df

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ
...,...,...,...,...,...,...
44995,60461435,Convert List<String> to string C# - asp.net - ...,<p>I am new to this and I am asking for help t...,<c#><asp.net><sql-server>,2020-02-29 02:22:18,LQ_CLOSE
44996,60461754,Does Python execute code from the top or botto...,<p>I am working on learning Python and was won...,<python>,2020-02-29 03:33:59,LQ_CLOSE
44997,60462001,how to change payment date in Azure?,<p>It looks like it costs 8 days per month in ...,<azure><billing>,2020-02-29 04:34:16,LQ_CLOSE
44998,60465318,how to implement fill in the blank in Swift,"<p>""I _____ any questions.""</p>\n\n<p>I want t...",<ios><swift>,2020-02-29 12:50:43,LQ_CLOSE


In [4]:
train_df['Target'] = train_df['Y']
val_df['Target'] = val_df['Y']

train_df.replace({'Target':{'LQ_EDIT':0,'HQ':1, 'LQ_CLOSE':2}}, inplace=True)
val_df.replace({'Target':{'LQ_EDIT':0,'HQ':1, 'LQ_CLOSE':2}}, inplace=True)

In [5]:
#creating training ang testing sets
train_set = train_df[['Body','Target']].sample(frac=1)
test_set = val_df[['Body','Target']].sample(frac=1)
train_set

,Body,Target
1716,<p>I try to use date() function</p>\n\n<p>But ...,2
37861,"# TL;DR:\r\n\r\nSurely somebody has written, a...",0
4020,<p>[!] Unable to satisfy the following require...,1
40532,<p>Is there a way to append a variable amount ...,2
38597,<p>i am very confuse about this simple code.\n...,2
...,...,...
34898,<p>I did the exact same thing in another part ...,2
3895,<p>I am very new to this IP block thing so ple...,2
31521,<p><strong>How can we load test Firebase Realt...,1
1327,"i have a code :\r\nSELECT pegawai.Nama, pegawa...",0


In [6]:
print(f"Total train samples: {len(train_set)}")
print(f"Total test samples: {len(test_set)}")
print(f"Total  samples: {len(train_set)+len(test_set)}")

Total train samples: 45000
Total test samples: 15000
Total  samples: 60000


# 3. Попередня оцінка точності моделі за допомогою методу Naive Bayes.

In [7]:
random_index=random.randint(0, len(train_df)-4)

for row in train_set[['Body','Target']][random_index:random_index+4].itertuples():
    print(row)

Pandas(Index=4154, Body='<p>In the default <code>_Layout.cshtml</code> file, scripts are defined in "environment"s like so:</p>\n\n<pre><code>&lt;environment names="Development"&gt;\n     &lt;script src="~/lib/jquery/dist/jquery.js"&gt;&lt;/script&gt;\n     &lt;script src="~/lib/bootstrap/dist/js/bootstrap.js"&gt;&lt;/script&gt;\n     &lt;script src="~/js/site.js" asp-append-version="true"&gt;&lt;/script&gt;\n&lt;/environment&gt;\n&lt;environment names="Staging,Production"&gt;\n     &lt;script src="https://ajax.aspnetcdn.com/ajax/jquery/jquery-2.1.4.min.js"\n                    asp-fallback-src="~/lib/jquery/dist/jquery.min.js"\n                    asp-fallback-test="window.jQuery"&gt;\n     &lt;/script&gt;\n     &lt;script src="https://ajax.aspnetcdn.com/ajax/bootstrap/3.3.5/bootstrap.min.js"\n                    asp-fallback-src="~/lib/bootstrap/dist/js/bootstrap.min.js"\n                    asp-fallback-test="window.jQuery &amp;&amp; window.jQuery.fn &amp;&amp; window.jQuery.fn.moda

In [8]:
from sklearn.model_selection import train_test_split

train_sentences, test_sentences, train_labels,test_labels = train_test_split(
    train_set['Body'].to_numpy(),
    train_set['Target'].to_numpy(),
    test_size=0.1, 
    random_state=49
)

In [9]:
train_sentences.dtype, train_labels.dtype

(dtype('O'), dtype('int64'))

In [10]:
len(train_sentences), len(test_sentences)

(40500, 4500)

In [11]:
avg_length = round(sum([len(i.split()) for i in train_set['Body']])/len(train_set))
avg_length

111

In [18]:
text_vectorizer.adapt(train_sentences)
new_df = pd.DataFrame(data = train_df)

In [19]:
words = text_vectorizer.get_vocabulary()

In [20]:
from tensorflow.keras.layers import Embedding

In [21]:
max_vocab_length = 20000

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=avg_length)

text_vectorizer.adapt(train_set['Body'])

text_vectorizer(train_set['Body'])

<tf.Tensor: shape=(45000, 111), dtype=int64, numpy=
array([[   20,   138,     3, ...,     0,     0,     0],
       [18256,  6520,  1548, ...,   112,     1, 16892],
       [   29,   629,     3, ...,     0,     0,     0],
       ...,
       [ 3543,    27,   139, ..., 10137,  6863,    18],
       [    4,    15,     5, ...,     0,     0,     0],
       [   20,  4892,    73, ...,     0,     0,     0]])>

In [22]:
embedding = layers.Embedding(input_dim=max_vocab_length, 
                             output_dim=128, 
                             embeddings_initializer="uniform",
                             input_length=avg_length) 

embedding(text_vectorizer(train_set['Body']))

2021-12-13 12:01:32.075857: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2557440000 exceeds 10% of free system memory.


<tf.Tensor: shape=(45000, 111, 128), dtype=float32, numpy=
array([[[ 0.04996167,  0.01361753,  0.03604818, ...,  0.0222998 ,
         -0.00358267,  0.00627307],
        [ 0.00773716,  0.03048385, -0.04259297, ...,  0.00117772,
          0.03735575, -0.02950165],
        [ 0.00183811, -0.03810859, -0.04759128, ..., -0.02788651,
          0.00627958,  0.04327207],
        ...,
        [-0.03702985, -0.03786867, -0.00538582, ...,  0.04158168,
          0.03826476, -0.0184123 ],
        [-0.03702985, -0.03786867, -0.00538582, ...,  0.04158168,
          0.03826476, -0.0184123 ],
        [-0.03702985, -0.03786867, -0.00538582, ...,  0.04158168,
          0.03826476, -0.0184123 ]],

       [[ 0.00515758, -0.00813674,  0.04201993, ..., -0.04290581,
          0.02706405, -0.00508337],
        [-0.03906371, -0.02088499,  0.04849618, ..., -0.01512618,
          0.02137106,  0.01337464],
        [ 0.04508109,  0.04796636, -0.04545671, ...,  0.03655381,
         -0.04055475, -0.04701228],
        

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([("tfidf", TfidfVectorizer()), ("clf", MultinomialNB())])

model_0.fit(train_set['Body'], train_set['Target'])

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
baseline_score = model_0.score(test_set['Body'], test_set['Target'])
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 68.71%


# 4. Вирішення задачі класифікації за допомогою методу 1D Convolutional model.

In [25]:
tf.random.set_seed(49)

from tensorflow.keras import layers

In [26]:
model_5embedding = layers.Embedding(input_dim=10000, 
                                    output_dim=128, 
                                    embeddings_initializer="uniform", 
                                    input_length=15, 
                                    name="embedding_5")

In [27]:
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x= model_5embedding(x)
x= layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x= layers.GlobalMaxPool1D()(x)

outputs=layers.Dense(1,activation="sigmoid")(x)

model_5 = tf.keras.Model(inputs,outputs, name="model_5_Conv1D")

In [28]:
model_5.compile(
            loss="binary_crossentropy", 
            optimizer=tf.keras.optimizers.Adam(), 
            metrics=["accuracy"]
)

In [29]:
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 111)              0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 111, 128)          1280000   
                                                                 
 conv1d (Conv1D)             (None, 107, 32)           20512     
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)              

In [34]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(128)(x)
x = layers.Dense(128, activation="sigmoid")(x) 
outputs = layers.Dense(3, activation="softmax")(x)
model_gru = tf.keras.Model(inputs, outputs, name="model_GRU")

In [35]:
model_gru.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_gru_history = model_gru.fit(train_set['Body'], train_set['Target'],
                              epochs=10,
                              validation_data=(test_set['Body'], test_set['Target']))

In [ ]:
model_gru_pred_probs = model_gru.predict(test_set['Body'])

In [ ]:
model_gru_preds = tf.squeeze(tf.argmax(model_gru_pred_probs, axis = 1))
model_gru_preds

# 5. Графічне відображення тренування моделі.

In [ ]:
from matplotlib import pyplot as plt

def plot_history(history): 
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()